In [5]:
import pandas as pd 
import numpy as np

In [6]:
data=pd.read_csv("DecisionTree.csv")
data.head()

,Unnamed: 0,Outlook,Temp,Humidity,Wind,Decision
0,0,sunny,85,85,weak,No
1,1,sunny,80,90,strong,No
2,2,overcast,83,78,weak,yes
3,3,rainy,70,96,weak,yes
4,4,rainy,68,80,weak,yes


In [7]:
# inputs=data.drop('Decision',axis='coulmns')
inputs=data[['Outlook','Temp','Humidity','Wind']]
target=data['Decision']


In [8]:
#to change words to numbers
from sklearn.preprocessing import LabelEncoder

In [9]:
no_outlook=LabelEncoder()
no_temp=LabelEncoder()
no_humidity=LabelEncoder()
no_wind=LabelEncoder()

In [11]:
 inputs.loc[:, 'no_outlook'] = no_outlook.fit_transform(inputs['Outlook'])
 inputs.loc[:, 'no_temp'] = no_temp.fit_transform(inputs['Temp'])
 inputs.loc[:, 'no_humidity'] = no_humidity.fit_transform(inputs['Humidity'])
 inputs.loc[:, 'no_wind'] = no_wind.fit_transform(inputs['Wind'])
inputs.head()


,Outlook,Temp,Humidity,Wind,no_outlook,no_temp,no_humidity,no_wind
0,sunny,85,85,weak,2,11,5,1
1,sunny,80,90,strong,2,8,6,0
2,overcast,83,78,weak,0,10,3,1
3,rainy,70,96,weak,1,4,8,1
4,rainy,68,80,weak,1,2,4,1


In [12]:
inputs_no=inputs.drop(['Outlook','Temp','Humidity','Wind'],axis=1)
inputs_no

,no_outlook,no_temp,no_humidity,no_wind
0,2,11,5,1
1,2,8,6,0
2,0,10,3,1
3,1,4,8,1
4,1,2,4,1
5,1,1,1,0
6,0,0,0,0
7,2,6,7,1
8,2,3,1,1
9,1,7,4,1


In [13]:
from collections import Counter
def calculate_entropy(data):
    labels=data['Decision']
    value_counts=Counter(labels)
    total_count=len(labels)
    entropy=0
    for label in value_counts:
        propability=value_counts[label]/total_count
        entropy-=propability*np.log2(propability)
    return entropy
    print(entropy)

In [14]:
def calculate_information_gain(data,attribute):
    entropy_before=calculate_entropy(data)
    unique_values_of_attribute=data[attribute].unique()
    entropy_after=0

    for value in unique_values_of_attribute:
        subset=data[data[attribute]==value]
        prob_value=len(subset)/len(data)
        entropy_after+=prob_value*calculate_entropy(subset)

    return entropy_before-entropy_after
    print(entropy_before-entropy_after)
        

In [ ]:
#def select_best_attribute(data, attributes):
#    information_gains = [(attribute, information_gain(data, attribute)) for attribute in attributes]
 #   best_attribute, best_gain = max(information_gains, key=lambda x: x[1])
  #  return best_attribute

In [15]:
def select_best_attribute(data, attributes):
    best_attribute = None
    best_gain = -1  # Initialize with a very low value

    for attribute in attributes:
        gain =calculate_information_gain(data, attribute)
        if gain > best_gain:
            best_gain = gain
            best_attribute = attribute

    return best_attribute
    print(best_attribute)


In [17]:
def id3(data, attributes):
    if len(np.unique(data['Decision'])) <= 1:
        return np.unique(data['Decision'])[0]

    elif len(data) == 0:
        return np.unique(data['Decision'])[np.argmax(np.unique(data['Decision'], return_counts=True)[1])]

    elif len(attributes) == 0:
        return np.unique(data['Decision'])[np.argmax(np.unique(data['Decision'], return_counts=True)[1])]

    else:
        best_attribute = select_best_attribute(data, attributes)
        tree = {best_attribute: {}}
        for value in data[best_attribute].unique():
            sub_data = data[data[best_attribute] == value]
            sub_attributes = [attr for attr in attributes if attr != best_attribute]
            subtree = id3(sub_data, sub_attributes)
            tree[best_attribute][value] = subtree
        return tree

# Build the decision tree
attributes = data.columns[1:-1]  # Exclude the 'Decision' column
decision_tree = id3(data, attributes)

# Print the decision tree
print("Decision Tree:")
print(decision_tree)

        

        

Decision Tree:
{'Temp': {85: 'No', 80: 'No', 83: 'yes', 70: 'yes', 68: 'yes', 65: 'no', 64: 'yes', 72: {'Outlook': {'sunny': 'no', 'overcast': 'yes'}}, 69: 'yes', 75: 'yes', 81: 'yes', 71: 'no'}}


In [38]:
# Define the C4.5 algorithm
def c45(data, attributes, parent_data=None):
    # If all instances have the same classification, return that classification
    if len(np.unique(data['Decision'])) <= 1:
        return np.unique(data['Decision'])[0]

    # If the dataset is empty, return the classification of the majority class in the parent dataset
    elif len(data) == 0:
        return np.unique(parent_data['Decision'])[np.argmax(np.unique(parent_data['Decision'], return_counts=True)[1])]

    # If there are no more attributes to split on, return the classification of the majority class
    elif len(attributes) == 0:
        return np.unique(data['Decision'])[np.argmax(np.unique(data['Decision'], return_counts=True)[1])]

    else:
        best_attribute = select_best_attribute(data, attributes)
        tree = {best_attribute: {}}
        for value in data[best_attribute].unique():
            sub_data = data[data[best_attribute] == value]
            sub_attributes = [attr for attr in attributes if attr != best_attribute]
            subtree = c45(sub_data, sub_attributes, data)
            tree[best_attribute][value] = subtree
        return tree

# Build the C4.5 decision tree
attributes = data.columns[1:-1]  # Exclude the 'Decision' column
c45_decision_tree = c45(data, attributes)

# Print the C4.5 decision tree
print("C4.5 Decision Tree:")
print(c45_decision_tree)


C4.5 Decision Tree:
{'Temp': {85: 'No', 80: 'No', 83: 'yes', 70: 'yes', 68: 'yes', 65: 'no', 64: 'yes', 72: {'Outlook': {'sunny': 'no', 'overcast': 'yes'}}, 69: 'yes', 75: 'yes', 81: 'yes', 71: 'no'}}


In [19]:
import pandas as pd
import numpy as np

# Define the dataset
data = pd.read_csv('DecisionTree.csv')

# Filter out non-numeric attributes (excluding 'Decision')
attributes = [col for col in data.columns if col != 'Decision' and np.issubdtype(data[col].dtype, np.number)]

# Define a function to calculate the Gini impurity
def gini_impurity(data):
    labels = data['Decision']
    total_count = len(labels)
    impurity = 1.0
    for label in np.unique(labels):
        probability = len(labels[labels == label]) / total_count
        impurity -= probability**2
    return impurity
# Define a function to calculate the Gini index for a split
def gini_index(data, attribute, value):
    left_subset = data[data[attribute] <= value]
    right_subset = data[data[attribute] > value]

    left_impurity = gini_impurity(left_subset)
    right_impurity = gini_impurity(right_subset)

    total_count = len(data)
    weight_left = len(left_subset) / total_count
    weight_right = len(right_subset) / total_count

    return weight_left * left_impurity + weight_right * right_impurity

# Define a function to select the best attribute and split point
def select_best_split(data, attributes):
    best_split = None
    best_gini = 1.0

    for attribute in attributes:
        unique_values = data[attribute].unique()
        unique_values.sort()

        for i in range(1, len(unique_values)):
            value = (unique_values[i - 1] + unique_values[i]) / 2
            gini = gini_index(data, attribute, value)

            if gini < best_gini:
                best_gini = gini
                best_split = (attribute, value)

    return best_split
# Define the CART algorithm
def cart(data, attributes):
    # If all instances have the same classification, return that classification
    if len(np.unique(data['Decision'])) == 1:
        return np.unique(data['Decision'])[0]

    # If there are no more attributes to split on, return the majority class
    if len(attributes) == 0:
        return np.unique(data['Decision'])[np.argmax(np.unique(data['Decision'], return_counts=True)[1])]

    # Otherwise, find the best split and create subtrees
    best_split = select_best_split(data, attributes)
    best_attribute, best_value = best_split

    left_data = data[data[best_attribute] <= best_value]
    right_data = data[data[best_attribute] > best_value]

    left_subtree = cart(left_data, [attr for attr in attributes if attr != best_attribute])
    right_subtree = cart(right_data, [attr for attr in attributes if attr != best_attribute])

    return {'attribute': best_attribute, 'value': best_value, 'left': left_subtree, 'right': right_subtree}

# Build the CART decision tree
cart_decision_tree = cart(data, attributes)

# Print the CART decision tree
print("CART Decision Tree:")
print(cart_decision_tree)

CART Decision Tree:
{'attribute': 'Unnamed: 0', 'value': 1.5, 'left': 'No', 'right': {'attribute': 'Temp', 'value': 73.5, 'left': {'attribute': 'Humidity', 'value': 67.5, 'left': 'yes', 'right': 'yes'}, 'right': 'yes'}}
